# 실습: 전국 무인 교통 단속 카메라 표준 데이터 `JSON` 데이터 `pandas`로 호출하기
- 데이터: 전국 무인 교통 단속 카메라 표준 데이터
- 제공기관: 공공데이터 활용 지원센터

In [1]:
# 필요 라이브러리 로딩
import pandas as pd
import json # json을 다루기 위한 라이브러리
import requests # API 서버 쪽에 request 보낼 떄 사용하는 라이브러리

## 1. Open API 호출 시 필요한 정보 입력

In [26]:
key = 'uioT%2FBViER2jFJYOplu6dlifMIcwqlVd6UPeyyugDejfla0%2FofPrk1fUGylGABlaq9ZzYWSFZrn1CQRtZhsb1g%3D%3D' # 발급받은 '일반 인증키' 복사 후 입력
pageNo = 1 # 요청변수 란에서 확인 : 페이지 번호 입력
row = 100 # 요청변수 란에서 확인 : 한 페이지 결과 수 입력
# type은 json으로 고정할 것이기 때문에 변수로 잡지 않음

'''
full url 입력 : `주소` + `?servicekey={키(위 변수)}` 
                       + `&pageNo={페이지번호(위 변수)}` 
                       + `&numOfRows={한 페이지 출력 결과 수}` 
                       + `&type=json`

'''
# full url 입력: f' '로 표현 
url = f'http://api.data.go.kr/openapi/tn_pubr_public_unmanned_traffic_camera_api?serviceKey={key}&pageNo={pageNo}&numOfRows={row}&type=json'

response = requests.get(url) # full url을 requests 라이브러리를 통해 호출

# 한글 출력 시, encoding이 필요한 response encoding = 'utf-8' # utf-8로 설정
response.encoding = 'utf-8' # utf-8로 설정

print(response.text[0:400]) # response 객체의 text 출력해보기

{"response":{"header":{"resultCode":"00","resultMsg":"NORMAL_SERVICE","type":"json"},"body":{"items":[{"mnlssRegltCameraManageNo":"P-7739","ctprvnNm":"경기도","signguNm":"고양시","roadKnd":"시도","roadRouteNo":"","roadRouteNm":"킨텍스로","roadRouteDrc":"3","rdnmadr":"","lnmadr":"경기도 고양시 일산서구 대화동 2603","latitude":"37.66585766","longitude":"126.7510974","itlpc":"일산서구 대화동 2603 (한화꿈에그린 106동앞)","regltSe":"4","lmtt


## 2. 결과 객체에 있는 데이터(json) 조회

In [12]:
resulting_dict = json.loads(response.text) # json을 dict 타입으로 바꾸는 loads 함수 
# print(resulting_dict['header'])
resulting_dict.keys() # dictionary 키 확인 --> 중첩 딕셔너리임

dict_keys(['response'])

In [13]:
# 중첩 딕셔너리의 키 확인
resulting_dict['response'].keys() 

dict_keys(['header', 'body'])

In [14]:
resulting_dict['response']['body'].keys() # 우리는 그중에 item 안에 있는 정보에 관심이 있음

dict_keys(['items', 'totalCount', 'numOfRows', 'pageNo'])

In [15]:
# resulting_dict['response']['body']['item'] # 이렇게 하면 너무 길어서, 하나만 뽑아보자.

In [17]:
# 첫번째 아이템 호출
resulting_dict['response']['body']['items'][0]

{'mnlssRegltCameraManageNo': 'P-7739',
 'ctprvnNm': '경기도',
 'signguNm': '고양시',
 'roadKnd': '시도',
 'roadRouteNo': '',
 'roadRouteNm': '킨텍스로',
 'roadRouteDrc': '3',
 'rdnmadr': '',
 'lnmadr': '경기도 고양시 일산서구 대화동 2603',
 'latitude': '37.66585766',
 'longitude': '126.7510974',
 'itlpc': '일산서구 대화동 2603 (한화꿈에그린 106동앞)',
 'regltSe': '4',
 'lmttVe': '0',
 'regltSctnLcSe': '',
 'ovrspdRegltSctnLt': '',
 'prtcareaType': '',
 'installationYear': '2019',
 'institutionNm': '경기도 고양시',
 'phoneNumber': '031-8075-2578',
 'referenceDate': '2023-11-24',
 'insttCode': '3940000'}

## 3. 특정 정보만 사용해서 데이터프레임으로 변환

In [23]:
# 주소, 카메라 위치, 위도, 경도 정보만 출력

# 각 관련 정보를 받을 리스트 생성
address_list = [] # 주소 리스트 생성 
location_list = [] # cctv 위치 리스트 생성
latitude_list = [] # 위도 리스트 생성
longitude_list = [] # 경도 리스트 생성

# dictionary속 데이터 정보 입력
# 정보를 뽑았던게, resultin_dict 안의 response 안의 body 안의 item(중점 리스트 명시)
for tmp in resulting_dict['response']['body']['items']: # 첫번째 tmp는 위에 출력한 [0]번째 item
    address_list.append(tmp['lnmadr'])
    location_list.append(tmp['itlpc']) 
    latitude_list.append(tmp['latitude']) # [0]에서 얻은 key 값을 참조해서 접근할 수 있게 하기
    longitude_list.append(tmp['longitude']) # append해서 각 리스트에 관련된 정보를 담기

print(address_list[:10]) # 100개를 호출해서 100개가 있겠지만, 10개만 불러와본다. 
print(location_list[:10])
print(longitude_list[:10])                   

['경기도 고양시 일산서구 대화동 2603', '경기도 고양시 일산서구 대화동 2603-2', '경기도 고양시 일산서구 대화동 1099-7', '경기도 고양시 일산서구 대화동 2734', '경기도 고양시 일산서구 대화동 2321', '경기도 고양시 일산서구 대화동 2321', '경기도 고양시 일산서구 대화동 2288-7', '경기도 고양시 일산서구 대화동 2307-15', '경기도 고양시 일산서구 가좌동 360-1', '경기도 고양시 일산서구 가좌동 391-1']
['일산서구 대화동 2603 (한화꿈에그린 106동앞)', '일산서구 대화동 2603-2 (한화꿈에그린 102동앞)', '대화동 1099-7 현대모터스 스튜디오 건너편', '대화동 2734번지 한국지역난방공사 열공급시설 삼거리', '일산서구 중앙로 1600 일산서구청(신청사) 1(대화동2428 일산서구청 신청사 옆)', '일산서구 중앙로 1600 일산서구청(신청사) 2', '일산서구 대화동 2288-7 (문촌19단지 사거리) (5대)', '일산서구 대화동 2307-15 (대방디엠시티 102동 옆) (4대)', '가좌동 360-1송포초교 정문(가좌동 355-1 송포초등학교 정문앞)', '가좌동 391-1 송포초교 후문(가좌동 355-1 송포초등학교 후문앞)']
['126.7510974', '126.7510974', '126.7479285', '126.7404425', '126.745242', '126.745242', '126.7509003', '126.7484978', '126.7220093', '126.7201958']


In [24]:
# dataframe으로 변환
df = pd.DataFrame({ # 2. 데이터프레임으로 변환 작업
    'address': address_list, # 1. 키와 밸류의 쌍으로 묶어주어서
    'location': location_list,  
    'latitude': latitude_list,
    'longitude': longitude_list
})

df # 총 100행임을 출력 결과를 통해 확인(100rows)

,address,location,latitude,longitude
0,경기도 고양시 일산서구 대화동 2603,일산서구 대화동 2603 (한화꿈에그린 106동앞),37.66585766,126.7510974
1,경기도 고양시 일산서구 대화동 2603-2,일산서구 대화동 2603-2 (한화꿈에그린 102동앞),37.66585766,126.7510974
2,경기도 고양시 일산서구 대화동 1099-7,대화동 1099-7 현대모터스 스튜디오 건너편,37.6651343,126.7479285
3,경기도 고양시 일산서구 대화동 2734,대화동 2734번지 한국지역난방공사 열공급시설 삼거리,37.66213291,126.7404425
4,경기도 고양시 일산서구 대화동 2321,일산서구 중앙로 1600 일산서구청(신청사) 1(대화동2428 일산서구청 신청사 옆),37.67791456,126.745242
...,...,...,...,...
95,경기도 고양시 일산서구 대화동 2607,대화동 2607 킨텍스 지하차도 부근(대화동 2600-3 킨텍스 지하차도 주변),37.66624826,126.7459032
96,경기도 고양시 일산서구 대화동 2306-2,대화동 2306-2 (킨텍스북문1),37.671585,126.7451536
97,경기도 고양시 일산서구 대화동 2202-2,일산서구 대화동 2202-2 (우리은행앞),37.67582142,126.749069
98,경기도 고양시 일산서구 대화동 2600-6,대화동 2600-6 (현대자동차 정문),37.66733286,126.7482931
